In [55]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/opt/anaconda3/envs/ml_basic/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <54A1AE05-1E14-3DA2-A8D0-062134694298> /opt/anaconda3/envs/ml_basic/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' 

In [49]:


# Download Apple stock data
aapl = yf.Ticker("AAPL")
# Get historical data from 2016 (to match the existing dataset timeframe)
data = aapl.history(start='2000-01-01', end='2025-04-14')
# Reset index to make date a column
data = data.reset_index()
# Add symbol column to match existing df format
data['symbol'] = 'AAPL'
# Reorder columns to match existing df format
data = data[['Date', 'symbol', 'Open', 'Close', 'Low', 'High', 'Volume']]
# Rename columns to match existing df format
data.columns = ['date', 'symbol', 'open', 'close', 'low', 'high', 'volume']


data['date'] = pd.to_datetime(data['date'])
data = data.sort_values('date')
# Find the first index where 'low' becomes >= 1
first_valid_idx = data[data['low'] <= 1.0].index[-1]

# Filter the dataframe to keep only records from that point onwards
df = data.loc[first_valid_idx:].reset_index(drop=True)
df['log_close'] = np.log(df['close'])


# Compute close_diff and close_pct_change
df['close_diff'] = df['log_close'].diff()
# Fill the first NaN value with 0
# df['close_diff'] = df['close_diff'].fillna(0)
df['close_pct_change'] = df['log_close'].pct_change() * 100  # Percentage change of close

In [50]:

# Feature Engineering
def create_features(df):
    # Lagged features
    for lag in [1, 2, 3, 5]:
        df[f'close_pct_change_lag{lag}'] = df['close_pct_change'].shift(lag)
        # df[f'close_lag{lag}'] = df['close'].shift(lag)
        df[f'volume_lag{lag}'] = df['volume'].shift(lag)
        # df[f'close_diff_lag{lag}'] = df['close_diff'].shift(lag)
    
    # Rolling statistics
    # df['close_rolling_mean_5'] = df['close'].rolling(window=5).mean()
    df['close_rolling_std_5'] = df['close'].rolling(window=5).std()
    df['volume_rolling_mean_5'] = df['volume'].rolling(window=5).mean()
    
    # Technical indicators
    df['rsi'] = compute_rsi(df['close'], 14)
    # df['sma_20'] = df['close'].rolling(window=20).mean()
    # df['ema_12'] = df['close'].ewm(span=12, adjust=False).mean()
    
    return df

def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Create features
data = create_features(df)

In [51]:
data.tail()

,date,symbol,open,close,low,high,volume,log_close,close_diff,close_pct_change,...,volume_lag1,close_pct_change_lag2,volume_lag2,close_pct_change_lag3,volume_lag3,close_pct_change_lag5,volume_lag5,close_rolling_std_5,volume_rolling_mean_5,rsi
5008,2025-04-07 00:00:00-04:00,AAPL,177.199997,181.460007,174.619995,194.149994,160466300,5.201035,-0.037426,-0.714445,...,125910900.0,-1.792840,103419000.0,0.057903,35905900.0,0.357104,65299300.0,19.471502,92422960.0,28.387551
5009,2025-04-08 00:00:00-04:00,AAPL,186.699997,172.419998,169.210007,190.339996,120859500,5.149933,-0.051102,-0.982534,...,160466300.0,-1.424130,125910900.0,-1.792840,103419000.0,0.088106,36412700.0,20.200949,109312320.0,22.809248
5010,2025-04-09 00:00:00-04:00,AAPL,171.949997,198.850006,171.889999,200.610001,184395900,5.292551,0.142617,2.769307,...,120859500.0,-0.714445,160466300.0,-1.424130,125910900.0,0.057903,35905900.0,12.554015,139010320.0,42.670386
5011,2025-04-10 00:00:00-04:00,AAPL,189.070007,190.419998,183.000000,194.779999,121880000,5.249232,-0.043319,-0.818483,...,184395900.0,-0.982534,120859500.0,-0.714445,160466300.0,-1.792840,103419000.0,9.935835,142702520.0,37.141012
5012,2025-04-11 00:00:00-04:00,AAPL,186.100006,198.149994,186.059998,199.539993,87435900,5.289024,0.039792,0.758056,...,121880000.0,2.769307,184395900.0,-0.982534,120859500.0,-1.424130,125910900.0,11.311182,135007520.0,40.058122


In [52]:
# Prepare data for modeling
data = data.dropna()  # Remove rows with NaN values
features = [col for col in data.columns if col not in ['date', 'symbol', 'open','close','low', 'high', 'log_close', 'close_diff', 'close_pct_change']]
X = data[features]
y = data['close_pct_change']  # Target variable
# y = data['close_pct_change'].shift(-1)  # Predict next day's close_pct_change
# # Drop the last row since it has NaN target value
# X = X[:-1]
# y = y[:-1]

# Split data (80% train, 20% test, preserve time order)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse:.6f}, R^2 Score: {r2:.6f}')

# Predict next day's close_pct_change
last_row = data.iloc[-1:]
last_features = last_row[features]
last_features_scaled = scaler.transform(last_features)
next_day_pred = model.predict(last_features_scaled)[0]
print(f'Predicted close_pct_change for next day: {next_day_pred:.6f}')

# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print('\nFeature Importance:')
print(feature_importance)

# # Save model and scaler
# import joblib
# joblib.dump(model, 'rf_model.pkl')
# joblib.dump(scaler, 'scaler.pkl')

Mean Squared Error on Test Set: 0.117132, R^2 Score: 0.028275
Predicted close_pct_change for next day: -0.470527

Feature Importance:
                  feature  importance
5   close_pct_change_lag3    0.258010
3   close_pct_change_lag2    0.122121
0                  volume    0.110314
1   close_pct_change_lag1    0.097421
7   close_pct_change_lag5    0.093864
11                    rsi    0.057595
8             volume_lag5    0.051525
2             volume_lag1    0.049817
10  volume_rolling_mean_5    0.044350
6             volume_lag3    0.043132
4             volume_lag2    0.039218
9     close_rolling_std_5    0.032631


In [54]:
# Prepare data for modeling
data = data.dropna()  # Remove rows with NaN values
features = [col for col in data.columns if col not in ['date', 'symbol', 'open','close','low', 'high', 'log_close', 'close_diff', 'close_pct_change']]
X = data[features]
y = data['close_pct_change']  # Target variable
# y = data['close_pct_change'].shift(-1)  # Predict next day's close_pct_change
# # Drop the last row since it has NaN target value
# X = X[:-1]
# y = y[:-1]

# Split data (80% train, 20% test, preserve time order)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse:.6f}, R^2 Score: {r2:.6f}')

# Predict next day's close_pct_change
last_row = data.iloc[-1:]
last_features = last_row[features]
last_features_scaled = scaler.transform(last_features)
next_day_pred = model.predict(last_features_scaled)[0]
print(f'Predicted close_pct_change for next day: {next_day_pred:.6f}')

# # Feature importance
# feature_importance = pd.DataFrame({
#     'feature': features,
#     'importance': model.feature_importances_
# }).sort_values('importance', ascending=False)
# print('\nFeature Importance:')
# print(feature_importance)

# # Save model and scaler
# import joblib
# joblib.dump(model, 'rf_model.pkl')
# joblib.dump(scaler, 'scaler.pkl')

Mean Squared Error on Test Set: 0.159312, R^2 Score: -0.321644
Predicted close_pct_change for next day: -0.454399
